# Pmax Helper - 共有キー版（社内用）

**APIキー設定不要！すぐ使えます**

## 使い方
1. 下のセルを上から順番に実行
2. LP URLを入力して「動画を生成」ボタンをクリック
3. 完成した動画をダウンロード

⚠️ **注意**: このノートブックには共有APIキーが含まれています。外部に公開しないでください。

## 1. APIキーの設定（自動）

In [ ]:
import os
from google.colab import userdata

# Colabのシークレットキーから取得
FAL_KEY = userdata.get('FAL_KEY')
LAMBDA_SECRET_KEY = "dummy_key"  # 固定値

# 環境変数に設定
os.environ['FAL_KEY'] = FAL_KEY
os.environ['LAMBDA_SECRET_KEY'] = LAMBDA_SECRET_KEY

print("✅ APIキーを設定しました（Colabシークレットから取得）")
print(f"   FAL_KEY: {FAL_KEY[:20]}..." if FAL_KEY else "   ❌ FAL_KEYが見つかりません")

## 2. セットアップ

In [ ]:
# 既存のディレクトリを削除して最新版をクローン
!rm -rf pmax-helper
!git clone https://github.com/KingHippopotamus/pmax-helper.git
%cd pmax-helper

# 最新版を取得（念のため）
!git pull origin main
!echo "✅ 最新コミット:" && git log -1 --oneline
!echo "\n📝 video_generator.py の最終更新:"
!git log -1 --oneline -- backend/services/video_generator.py

%cd backend

# 依存関係をインストール
!pip install -q flask flask-cors requests beautifulsoup4 Pillow fal-client python-dotenv ipywidgets moviepy

print("\n✅ セットアップ完了")

## 3. 動画生成ツール

下のセルを実行すると、入力フォームが表示されます。

In [ ]:
import sys
import os

# モジュールキャッシュをクリア（重要！）
for module in list(sys.modules.keys()):
    if 'services' in module or 'fal' in module:  # fal関連も削除
        del sys.modules[module]

sys.path.insert(0, '/content/pmax-helper/backend')

# ⬇️⬇️⬇️ 重要: インポート前に環境変数を設定 ⬇️⬇️⬇️
# 環境変数からAPIキーを取得して、すぐに設定
FAL_KEY = os.getenv('FAL_KEY')
LAMBDA_SECRET_KEY = os.getenv('LAMBDA_SECRET_KEY')

if FAL_KEY:
    os.environ['FAL_KEY'] = FAL_KEY
    print(f"✅ FAL_KEY設定完了: {FAL_KEY[:20]}...")
else:
    print("❌ FAL_KEYが見つかりません")
# ⬆️⬆️⬆️ ここまで移動 ⬆️⬆️⬆️

# この後にインポート（FAL_KEYが既に設定済み）
from services.page_analyzer import PageAnalyzer
from services.video_generator import VideoGenerator
from services.scraper import ImageScraper
from services.image_processor import ImageProcessor
import ipywidgets as widgets
from IPython.display import display, HTML, Video, Image
import requests

# デバッグ: 読み込まれたモジュールのパスを確認
import services.video_generator as vg_module
print(f"\n[DEBUG] video_generator.pyのパス: {vg_module.__file__}")

# ファイルの内容を確認（モデル名が含まれているか）
with open(vg_module.__file__, 'r') as f:
    content = f.read()
    if 'fal-ai/sora-2/image-to-video/pro' in content:
        print("[DEBUG] ✅ Pro版モデルが検出されました")
    elif 'fal-ai/sora-2/image-to-video' in content:
        print("[DEBUG] ⚠️ 通常版モデルが検出されました（古いバージョン）")
print()

# 出力エリア
output = widgets.Output()

# UI要素
url_input = widgets.Text(
    placeholder='https://example.com',
    description='LP URL:',
    layout=widgets.Layout(width='600px')
)

analyze_button = widgets.Button(
    description='🔍 ページを分析',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

generate_button = widgets.Button(
    description='🎬 動画を生成',
    button_style='success',
    layout=widgets.Layout(width='150px'),
    disabled=True
)

# 分析結果編集エリア（初期は非表示）
edit_area = widgets.VBox([])

# 各項目のテキストボックス
product_name_input = widgets.Text(description='商材名:', layout=widgets.Layout(width='600px'))
target_input = widgets.Text(description='ターゲット:', layout=widgets.Layout(width='600px'))
catchphrase_input = widgets.Text(description='キャッチコピー:', layout=widgets.Layout(width='600px'))
benefit1_input = widgets.Text(description='ベネフィット1:', layout=widgets.Layout(width='600px'))
benefit2_input = widgets.Text(description='ベネフィット2:', layout=widgets.Layout(width='600px'))
offer_input = widgets.Text(description='オファー:', layout=widgets.Layout(width='600px'))
cta_input = widgets.Text(description='CTA:', layout=widgets.Layout(width='600px'))

# キャラクター画像プレビュー専用エリア
character_preview = widgets.Output()

# 画像カスタマイズUI
image_url_input = widgets.Text(
    placeholder='https://example.com/character.jpg または 空欄でLP自動抽出',
    description='画像URL:',
    layout=widgets.Layout(width='600px')
)

upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='または画像選択'
)

image_preview = widgets.Output()

# 生成モード選択（NEW!）
generation_mode = widgets.RadioButtons(
    options=['シングル生成（1本）', 'バッチ生成（5本）'],
    value='シングル生成（1本）',
    description='生成モード:',
    layout=widgets.Layout(width='300px')
)

# アスペクト比選択ドロップダウン
aspect_ratio_dropdown = widgets.Dropdown(
    options=['16:9 (横長)', '9:16 (縦長)', '1:1 (正方形)'],
    value='16:9 (横長)',
    description='動画形式:',
    layout=widgets.Layout(width='300px')
)

# アスペクト比コンテナ（表示/非表示切り替え用）
aspect_ratio_container = widgets.VBox([
    widgets.HTML("<h3>🎞️ 動画形式</h3>"),
    aspect_ratio_dropdown
])

# グローバル変数
product_info = {}
character_image_url = ""
character_image_data = None
video_url = ""

def on_analyze_clicked(b):
    global product_info, character_image_url
    
    with output:
        output.clear_output()
        print("🔍 ページを分析しています...\n")
        
        try:
            analyzer = PageAnalyzer(LAMBDA_SECRET_KEY)
            result = analyzer.analyze_page(url_input.value)
            
            if 'error' in result:
                print(f"\n❌ エラー: {result['error']}")
                return
            
            product_info = result
            character_image_url = result.get('character_image_url', '')
            
            product_name_input.value = result.get('product_name', '')
            target_input.value = result.get('target_audience', '')
            catchphrase_input.value = result.get('catchphrase', '')
            benefit1_input.value = result.get('benefit1', '')
            benefit2_input.value = result.get('benefit2', '')
            offer_input.value = result.get('offer', '')
            cta_input.value = result.get('cta_text', '')
            
            with character_preview:
                character_preview.clear_output()
                if character_image_url:
                    print("🖼️ LP抽出キャラクター画像:")
                    display(HTML(f'<img src="{character_image_url}" width="300">'))
                else:
                    print("⚠️ キャラクター画像が見つかりませんでした")
            
            edit_area.children = [
                widgets.HTML("<h3>📝 分析結果（編集可能）</h3>"),
                character_preview,
                product_name_input,
                target_input,
                catchphrase_input,
                benefit1_input,
                benefit2_input,
                offer_input,
                cta_input
            ]
            
            output.clear_output()
            print("✅ 分析完了！必要に応じて内容を編集してください。")
            generate_button.disabled = False
            
        except Exception as e:
            print(f"\n❌ エラーが発生しました: {str(e)}")
            import traceback
            traceback.print_exc()

def on_image_url_changed(change):
    global character_image_url, character_image_data
    url = change['new'].strip()
    if url:
        character_image_url = url
        character_image_data = None
        with image_preview:
            image_preview.clear_output()
            print(f"✅ カスタム画像URL指定: {url[:50]}...")
            display(HTML(f'<img src="{url}" width="200">'))
    else:
        character_image_data = None

def on_file_uploaded(change):
    global character_image_data, character_image_url
    uploaded_files = change['new']
    if uploaded_files:
        filename = list(uploaded_files.keys())[0]
        file_info = uploaded_files[filename]
        character_image_data = file_info['content']
        character_image_url = None
        with image_preview:
            image_preview.clear_output()
            print(f"✅ 画像アップロード完了: {filename}")
            display(Image(data=character_image_data, width=200))

def on_mode_changed(change):
    """生成モードが変わったときの処理"""
    mode = change['new']
    
    if mode == 'シングル生成（1本）':
        aspect_ratio_container.children = [
            widgets.HTML("<h3>🎞️ 動画形式</h3>"),
            aspect_ratio_dropdown
        ]
        generate_button.description = '🎬 動画を生成'
    else:
        aspect_ratio_container.children = [
            widgets.HTML("<h3>🎞️ 動画形式（自動: 縦長×2、横長×2、正方形×1）</h3>")
        ]
        generate_button.description = '🎬 5本まとめて生成'

def display_single_result(result):
    """シングル生成結果を表示"""
    import tempfile
    from google.colab import files
    
    print("\n" + "="*60)
    print("✅ 動画生成完了！\n")
    
    if result.get('trimmed') or result.get('processed'):
        video_data = result.get('video_data')
        
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as tmp:
            tmp.write(video_data)
            video_path = tmp.name
        
        print(f"📹 動画ファイル: {video_path}\n")
        display(Video(video_path, width=640))
        
        download_button = widgets.Button(
            description='📥 動画をダウンロード',
            button_style='success'
        )
        
        def on_download(b):
            files.download(video_path)
        
        download_button.on_click(on_download)
        display(download_button)

def display_batch_results(result):
    """バッチ生成結果を表示"""
    import tempfile
    from google.colab import files
    
    summary = result['summary']
    videos = result['videos']
    
    print("\n" + "="*60)
    print("✅ バッチ動画生成完了！")
    print("="*60)
    print(f"成功: {summary['success']}/5")
    print(f"失敗: {summary['failed']}/5\n")
    
    success_videos = [v for v in videos if v and v['status'] == 'success']
    
    if not success_videos:
        print("❌ すべての動画生成に失敗しました")
        for error in summary['errors']:
            print(f"   - {error}")
        return
    
    video_paths = []
    
    for video in success_videos:
        index = video['index']
        name = video['name']
        aspect_ratio = video['aspect_ratio']
        video_data = video['video_data']
        
        with tempfile.NamedTemporaryFile(delete=False, suffix=f'_{name}.mp4') as tmp:
            tmp.write(video_data)
            video_path = tmp.name
            video_paths.append((video_path, name, aspect_ratio))
        
        print(f"\n📹 {name} ({aspect_ratio})")
        display(Video(video_path, width=400))
    
    print("\n" + "="*60)
    
    download_all_button = widgets.Button(
        description=f'📥 {len(success_videos)}本まとめてダウンロード',
        button_style='success',
        layout=widgets.Layout(width='300px')
    )
    
    def on_download_all(b):
        for video_path, name, aspect_ratio in video_paths:
            filename = f"character_video_{name}_{aspect_ratio.replace(':', 'x')}.mp4"
            import shutil
            shutil.copy(video_path, filename)
            files.download(filename)
        print(f"✅ {len(video_paths)}本の動画をダウンロードしました")
    
    download_all_button.on_click(on_download_all)
    display(download_all_button)
    
    if summary['errors']:
        print("\n⚠️ 一部の動画生成に失敗しました:")
        for error in summary['errors']:
            print(f"   - {error}")

def on_generate_clicked(b):
    global video_url, product_info
    
    product_info.update({
        'product_name': product_name_input.value,
        'target_audience': target_input.value,
        'catchphrase': catchphrase_input.value,
        'benefit1': benefit1_input.value,
        'benefit2': benefit2_input.value,
        'offer': offer_input.value,
        'cta_text': cta_input.value
    })
    
    with output:
        output.clear_output()
        
        is_batch = (generation_mode.value == 'バッチ生成（5本）')
        
        if is_batch:
            print("🎬 バッチ動画生成を開始します（5本並列）")
            print("   処理時間: 2-5分程度\n")
        else:
            print("🎬 動画を生成しています...（数分かかる場合があります）\n")
        
        try:
            if character_image_data:
                print("✅ アップロード画像を使用\n")
                image_data = character_image_data
            elif character_image_url:
                print(f"📥 画像をダウンロード中: {character_image_url[:80]}...")
                response = requests.get(character_image_url, timeout=10)
                response.raise_for_status()
                image_data = response.content
                print(f"✅ 画像ダウンロード完了 ({len(image_data)} bytes)\n")
            else:
                print("❌ キャラクター画像が指定されていません")
                return
            
            print("🖼️ 画像を前処理中...")
            processor = ImageProcessor()
            processed_image = processor.preprocess_image(image_data)
            print("✅ 画像前処理完了\n")
            
            generator = VideoGenerator()
            analyzer = PageAnalyzer(LAMBDA_SECRET_KEY)
            
            if is_batch:
                prompt = analyzer._generate_video_prompt(product_info, aspect_ratio="16:9")
                result = generator.generate_character_video_batch(
                    processed_image,
                    prompt=prompt
                )
                display_batch_results(result)
            else:
                selected_ratio = aspect_ratio_dropdown.value.split(' ')[0]
                print(f"🎞️ アスペクト比: {selected_ratio}\n")
                
                prompt = analyzer._generate_video_prompt(product_info, aspect_ratio=selected_ratio)
                result = generator.generate_character_video(
                    processed_image,
                    prompt=prompt,
                    aspect_ratio=selected_ratio
                )
                display_single_result(result)
                
        except Exception as e:
            error_msg = str(e)
            if 'コンテンツポリシー違反' in error_msg:
                print("\n" + "="*60)
                print("⚠️ コンテンツポリシー違反エラー")
                print("="*60)
                print(f"\n{error_msg}\n")
            else:
                print(f"\n❌ エラーが発生しました: {error_msg}")
            import traceback
            traceback.print_exc()

# イベントハンドラを登録
analyze_button.on_click(on_analyze_clicked)
image_url_input.observe(on_image_url_changed, names='value')
upload_button.observe(on_file_uploaded, names='value')
generation_mode.observe(on_mode_changed, names='value')
generate_button.on_click(on_generate_clicked)

# UIを表示
display(widgets.VBox([
    widgets.HTML("<h2>🎬 Pmax Helper - AI動画生成ツール（共有キー版）</h2>"),
    
    url_input,
    widgets.HBox([analyze_button]),
    
    edit_area,
    
    widgets.HTML("<hr><h3>📸 キャラクター画像の変更（任意）</h3>"),
    widgets.HTML("<p style='color: #666;'>LP分析で抽出された画像を使う場合は、以下を空欄のままにしてください</p>"),
    image_url_input,
    upload_button,
    image_preview,
    
    widgets.HTML("<hr><h3>⚙️ 生成モード</h3>"),
    generation_mode,
    
    widgets.HTML("<hr>"),
    aspect_ratio_container,
    
    widgets.HTML("<hr>"),
    widgets.HBox([generate_button]),
    output
]))

print("\n👆 上のフォームにLP URLを入力して、ボタンをクリックしてください")

---

## トラブルシューティング

### エラーが出る場合
- LP URLが正しいか確認
- セルを上から順番に実行したか確認

### 動画生成に時間がかかる
- fal-ai APIは通常2-5分かかります
- セッションがタイムアウトした場合は最初から再実行